In [4]:
!pip install fuzzywuzzy

In [226]:
#skillsfuture dataset
sf = pd.read_excel("C:/Users/valli/Google Drive/Dell/Semester 6/Capstone/Data/Enhanced construct - 16 sectors.xlsx")

In [350]:
sf['Sector'].unique()

array(['Design', 'Energy and Power', 'Engineering Services',
       'Financial Services', 'Healthcare', 'Intellectual Property',
       'Landscape', 'Media', 'Social Service',
       'Training and Adult Education', 'Wholesale Trade',
       'Workplace Safety and Health', 'Infocomm Technology',
       'Sea Transport', 'Accountancy', 'Built Environment'], dtype=object)

In [373]:
example1 = sf[sf["Sector"]=="Media"]['TSC Description'].head(1).to_string(index = False)

In [375]:
example2 = sf[sf["Sector"]=="Infocomm Technology"]['TSC Description'].head(1).to_string(index = False)

In [376]:
example3 = sf[sf["Sector"]=="Training and Adult Education"]['TSC Description'].head(1).to_string(index = False)

In [377]:
example4 = sf[sf["Sector"]=="Engineering Services"]['TSC Description'].head(1).to_string(index = False)

In [447]:
example5 = sf[sf["Sector"]=="Healthcare"]['TSC Description'].head(1).to_string(index = False)

In [ ]:
from Portal import *
# pd.set_option('display.width', 400)
# pd.set_option('display.max_columns', 10)


class CareersFuture(Portal):

    def __init__(self, name):
        super().__init__(name)

    def read_data(self):
        # Reads data
        df = pd.read_csv('mycareerfuture2020-09-15.csv', error_bad_lines=False)
        return df

    def clean_data(self):
        df = self.read_data()
        # df = df[df['Job Description'].str.contains('value')]
        df['Job Description'] = df['Job Description'].astype(str)
        df['Job Description'] = df['Job Description'].apply(lambda x: self.strip_html_tags(x))

        df['Industry'] = df['Industry'].astype(str)
        df['Industry'] = df['Industry'].apply(lambda x: self.str_to_literal(x))

        # df['Skills'] = df['Skills'].astype(str)

        df['Job Experience Required (years)'] = df['Job Experience Required (years)'].astype(int)

        df['Job Monthly Min Sal'] = df['Job Monthly Min Sal'].astype(int)
        df['Job Monthly Max Sal'] = df['Job Monthly Max Sal'].astype(int)

        return df

    def str_to_literal(self, text):
        try:
            # ast.literal_eval function converts takes in a string and converts it into a dictionary
            ls = super().str_to_literal(text)
            industry = []
            for i in ls:
                industry.append(i['category'])
            return industry
        except:
            pass

careersfuture = CareersFuture("careersfuture")


In [12]:
df = pd.DataFrame(careersfuture.clean_data())


C:\Users\valli\anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b' . '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [20]:
df['Industry_unlisted'] = df['Industry'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
df[df['Industry_unlisted'].unique()

In [32]:
contain_values = df[df['Industry_unlisted'].str.contains('Building and Construction,F&B')]

## Bert

In [35]:
!pip install transformers
!pip install pytorch

In [36]:
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np

%matplotlib inline

In [37]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model tokenizer (vocabulary)
model = BertModel.from_pretrained('bert-base-uncased')

In [74]:
sentence_samples = df['Job Description']
df_samples = pd.DataFrame(sentence_samples)

In [64]:
tokenized = df['Job Description'][:1000].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
check_idx = 4
print("======input sentence======")
print(df['Job Description'][check_idx])
breaken_words = []
for idx in tokenized[check_idx]:
    breaken_words.append(tokenizer.convert_ids_to_tokens(idx))
print("======after breaking======")
print(" ".join(breaken_words))
print("======Convert to IDs======")
print((tokenized[check_idx]))

In [66]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print(padded.shape)

(1000, 1917)


In [67]:
print(padded[0])
print(tokenizer.convert_ids_to_tokens(0))

[ 101 1996 3026 ...    0    0    0]
[PAD]


In [68]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded).long() 
attention_mask = torch.tensor(attention_mask)



In [69]:
with torch.no_grad():
    ## the output here is a tuple of (last_hidden_state, pooler_output)
    model_outputs = model(input_ids, attention_mask=attention_mask)

RuntimeError: The size of tensor a (1917) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
# get the last_hidden state
last_hidden_state = model_outputs[0].numpy()

## Model for Classfication

In [400]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline



In [401]:
df_model = df[['Job Description', 'Industry_unlisted']]

print(df_model['Job Description'].apply(lambda x: len(x.split(' '))).sum())

9707657


In [404]:
ind = ['Information Technology', 'Engineering', 'F&B', 'Banking and Finance', 'Accounting / Auditing / Taxation', 'Building and Construction', 'Logistics / Supply Chain', 'Others', 'Education and Training', 'Sales / Retail']

In [405]:
df_selected = df_model[df_model['Industry_unlisted'].isin(ind)]

In [407]:
# def print_plot(index):
#     example = df_selected[df_selected.index == index][['Job Description', 'Industry_unlisted']].values[0]
#     if len(example) > 0:
#         print(example[0])
#         print('Industry:', example[1])
# print_plot(10)

In [433]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = lemmatize_stemming(text)
    return text
    
df_selected['Job Description'] = df_selected['Job Description'].apply(clean_text)
# print_plot(10)

C:\Users\valli\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [434]:
df_selected['Job Description'].apply(lambda x: len(x.split(' '))).sum()

2783533

In [435]:
X = df_selected['Job Description']
y = df_selected['Industry_unlisted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [160]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfTransformer

# nb = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', MultinomialNB()),
#               ])
# nb.fit(X_train, y_train)


# from sklearn.metrics import classification_report
# y_pred = nb.predict(X_test)

# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred,target_names=ind))

accuracy 0.5855299036538811
                                  precision    recall  f1-score   support

          Information Technology       0.96      0.54      0.69       377
                     Engineering       0.92      0.15      0.26       456
                             F&B       0.89      0.43      0.58       379
             Banking and Finance       0.97      0.46      0.62       264
Accounting / Auditing / Taxation       0.81      0.28      0.42       692
       Building and Construction       0.90      0.83      0.86       466
        Logistics / Supply Chain       0.47      1.00      0.64      1923
                          Others       0.84      0.41      0.55       370
          Education and Training       0.00      0.00      0.00       280
                  Sales / Retail       1.00      0.02      0.03       294

                        accuracy                           0.59      5501
                       macro avg       0.78      0.41      0.47      5501
        

C:\Users\valli\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [398]:
df_selected.iloc[10605].shape

(2,)

In [162]:
# nb.predict(df_selected.iloc[10605])

array(['Information Technology', 'Banking and Finance'], dtype='<U32')

In [163]:
# nb.predict(df_selected.iloc[2701])

array(['Information Technology', 'Information Technology'], dtype='<U32')

In [436]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=ind))

accuracy 0.8180330848936557
                                  precision    recall  f1-score   support

          Information Technology       0.87      0.92      0.89       377
                     Engineering       0.84      0.71      0.77       456
                             F&B       0.78      0.84      0.81       379
             Banking and Finance       0.84      0.95      0.89       264
Accounting / Auditing / Taxation       0.85      0.61      0.71       692
       Building and Construction       0.91      0.94      0.93       466
        Logistics / Supply Chain       0.80      0.97      0.88      1923
                          Others       0.76      0.85      0.80       370
          Education and Training       0.88      0.05      0.09       280
                  Sales / Retail       0.74      0.70      0.72       294

                        accuracy                           0.82      5501
                       macro avg       0.83      0.75      0.75      5501
        

In [437]:
sgd.predict(df_selected.iloc[10605])

array(['Banking and Finance', 'Banking and Finance'], dtype='<U32')

In [451]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=ind))

C:\Users\valli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy 0.833484820941647
                                  precision    recall  f1-score   support

          Information Technology       0.86      0.88      0.87       377
                     Engineering       0.84      0.78      0.81       456
                             F&B       0.80      0.79      0.80       379
             Banking and Finance       0.91      0.95      0.93       264
Accounting / Auditing / Taxation       0.80      0.78      0.79       692
       Building and Construction       0.94      0.91      0.92       466
        Logistics / Supply Chain       0.89      0.91      0.90      1923
                          Others       0.81      0.80      0.80       370
          Education and Training       0.43      0.46      0.45       280
                  Sales / Retail       0.75      0.68      0.71       294

                        accuracy                           0.83      5501
                       macro avg       0.80      0.80      0.80      5501
         

In [452]:
logreg.predict(df_selected.iloc[10605])

array(['Banking and Finance', 'Banking and Finance'], dtype=object)

In [453]:
log_results = pd.DataFrame(X_test).reset_index()
Y_test = pd.DataFrame(y_test).reset_index()
Y_pred = pd.DataFrame(y_pred).reset_index()
log_results = pd.concat([log_results, Y_test['Industry_unlisted'], Y_pred[0]], axis = 1, ignore_index=True)

In [454]:
log_results.columns = ['Row', 'Job Description', 'Industry', 'Predicted']

In [ ]:
log_results[log_results['Industry'] != log_results['Predicted']]

In [456]:
logreg.predict([example1])
#Media

array(['Others'], dtype=object)

In [457]:
logreg.predict([example2])
#IT

array(['Sales / Retail'], dtype=object)

In [458]:
logreg.predict([example3])
#Education

array(['Education and Training'], dtype=object)

In [459]:
logreg.predict([example4])
#Engineering

array(['Engineering'], dtype=object)

In [460]:
logreg.predict([example5])
#Healthcare

array(['Information Technology'], dtype=object)

## Topic Modelling

In [331]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\valli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [338]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
df_tm = df[['Job Description', 'Industry_unlisted']]
processed = df_tm['Job Description'].map(preprocess)
processed[:10]

In [341]:
dictionary = gensim.corpora.Dictionary(processed)

In [342]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abl
1 accord
2 analyz
3 annual
4 aspect
5 assist
6 attend
7 candid
8 clean
9 code
10 complaint


In [344]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed]
bow_corpus[4310]

[(0, 3),
 (19, 1),
 (45, 1),
 (53, 1),
 (105, 4),
 (111, 4),
 (129, 1),
 (134, 1),
 (136, 1),
 (138, 4),
 (144, 3),
 (157, 1),
 (167, 1),
 (196, 1),
 (198, 6),
 (206, 1),
 (208, 2),
 (216, 4),
 (229, 1),
 (254, 2),
 (266, 2),
 (270, 1),
 (284, 1),
 (287, 2),
 (288, 2),
 (290, 1),
 (293, 1),
 (295, 1),
 (297, 2),
 (311, 4),
 (315, 1),
 (326, 1),
 (331, 1),
 (335, 2),
 (347, 1),
 (352, 2),
 (362, 1),
 (373, 1),
 (376, 2),
 (400, 1),
 (411, 1),
 (412, 1),
 (419, 1),
 (425, 2),
 (433, 1),
 (441, 2),
 (445, 4),
 (449, 1),
 (455, 1),
 (498, 2),
 (499, 2),
 (516, 3),
 (553, 1),
 (573, 1),
 (578, 1),
 (592, 2),
 (606, 2),
 (657, 2),
 (658, 1),
 (743, 1),
 (755, 4),
 (774, 1),
 (800, 2),
 (828, 1),
 (863, 2),
 (887, 1),
 (904, 1),
 (910, 3),
 (974, 1),
 (1003, 1),
 (1044, 1),
 (1070, 7),
 (1137, 1),
 (1145, 1),
 (1184, 1),
 (1203, 2),
 (1214, 1),
 (1215, 3),
 (1227, 1),
 (1261, 1),
 (1315, 1),
 (1591, 1),
 (1741, 2),
 (2022, 1),
 (2168, 1),
 (2822, 1),
 (2908, 1),
 (2909, 1),
 (2912, 1),
 (3248

In [345]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.02836967302244393),
 (1, 0.056512122877126675),
 (2, 0.07485220306391939),
 (3, 0.07976544675713589),
 (4, 0.08087816759178439),
 (5, 0.03244767513970287),
 (6, 0.06645605270427057),
 (7, 0.03329064886221295),
 (8, 0.07612410639169918),
 (9, 0.07250789405860913),
 (10, 0.20082338925800625),
 (11, 0.05950843122171499),
 (12, 0.05072703842597369),
 (13, 0.06718451253848687),
 (14, 0.1844223304558999),
 (15, 0.18247912067851063),
 (16, 0.049644970510950466),
 (17, 0.13635244079568923),
 (18, 0.055763617725171216),
 (19, 0.0490811259715086),
 (20, 0.062277519623150174),
 (21, 0.10079487309888514),
 (22, 0.07427661004717229),
 (23, 0.06992195039628246),
 (24, 0.07638356581507297),
 (25, 0.08934323997423581),
 (26, 0.07545194285225845),
 (27, 0.0673528854829313),
 (28, 0.06940618187857119),
 (29, 0.023383341787665032),
 (30, 0.10772174646367379),
 (31, 0.07167065287577812),
 (32, 0.2144541896435618),
 (33, 0.04170273477403491),
 (34, 0.094822729251375),
 (35, 0.047439786718102034),
 (

In [346]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [347]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"busi" + 0.015*"develop" + 0.014*"project" + 0.011*"client" + 0.010*"product" + 0.009*"custom" + 0.009*"market" + 0.008*"support" + 0.008*"strong" + 0.008*"servic"
Topic: 1 
Words: 0.024*"secur" + 0.015*"oper" + 0.015*"support" + 0.014*"network" + 0.012*"ensur" + 0.011*"perform" + 0.011*"system" + 0.009*"servic" + 0.009*"knowledg" + 0.009*"provid"
Topic: 2 
Words: 0.036*"data" + 0.014*"busi" + 0.011*"develop" + 0.009*"analyt" + 0.008*"product" + 0.008*"build" + 0.008*"process" + 0.008*"technolog" + 0.007*"model" + 0.007*"learn"
Topic: 3 
Words: 0.018*"product" + 0.014*"good" + 0.013*"abl" + 0.012*"oper" + 0.012*"perform" + 0.011*"equip" + 0.011*"custom" + 0.011*"ensur" + 0.008*"process" + 0.008*"duti"
Topic: 4 
Words: 0.011*"care" + 0.010*"provid" + 0.010*"research" + 0.009*"patient" + 0.008*"assist" + 0.008*"school" + 0.008*"nurs" + 0.008*"candid" + 0.008*"educ" + 0.007*"singapor"
Topic: 5 
Words: 0.016*"account" + 0.013*"custom" + 0.013*"ensur" + 0.013*"prepar"

In [348]:
unseen_document = 'Designing and implementing chatbots and its optimisation platforms which are used by millions of Shopee customers and internal chatbot operations\
Communicating with product managers and designers and taking responsibility for the development and testing of products\
Understanding the design and product requirements, whilst being familiar with the architecture and implementing solutions to highly complex and flexible features\
Maintaining and improving our high-quality in-house JavaScript libraries and toolsets ( Example: A complete React UI framework following our design guidelines)\
Participating in rigorous and candid code reviews with peers\
Participating in regular internal technology sharings and other regional tech events'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6524395942687988	 Topic: 0.033*"develop" + 0.018*"applic" + 0.016*"technolog" + 0.015*"softwar" + 0.014*"design"
Score: 0.2287287563085556	 Topic: 0.035*"design" + 0.028*"market" + 0.018*"develop" + 0.015*"product" + 0.013*"media"
Score: 0.10561554878950119	 Topic: 0.022*"busi" + 0.015*"develop" + 0.014*"project" + 0.011*"client" + 0.010*"product"


In [471]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"project" + 0.009*"site" + 0.008*"construct" + 0.006*"draw" + 0.006*"engin" + 0.006*"safeti" + 0.005*"children" + 0.005*"coordin" + 0.005*"design" + 0.005*"prepar"
Topic: 1 Word: 0.015*"market" + 0.009*"media" + 0.007*"content" + 0.007*"digit" + 0.007*"campaign" + 0.007*"social" + 0.006*"brand" + 0.005*"sale" + 0.005*"creativ" + 0.005*"design"
Topic: 2 Word: 0.007*"equip" + 0.006*"machin" + 0.005*"mainten" + 0.005*"product" + 0.004*"repair" + 0.004*"perform" + 0.004*"custom" + 0.004*"manufactur" + 0.004*"oper" + 0.004*"care"
Topic: 3 Word: 0.011*"food" + 0.006*"kitchen" + 0.005*"custom" + 0.005*"duti" + 0.005*"restaur" + 0.004*"order" + 0.004*"clean" + 0.004*"cook" + 0.004*"abl" + 0.004*"stock"
Topic: 4 Word: 0.006*"custom" + 0.005*"singtel" + 0.005*"thank" + 0.004*"hair" + 0.004*"servic" + 0.004*"shipment" + 0.004*"product" + 0.003*"ship" + 0.003*"day" + 0.003*"abl"
Topic: 5 Word: 0.006*"dental" + 0.004*"weld" + 0.003*"resum" + 0.003*"candid" + 0.003*"licenc" + 0.

In [473]:
j = 1
for i in [example1, example2, example3, example4, example5]:
    print("Example" + str(j)+ ": " + i)
    j+=1
    bow_vector = dictionary.doc2bow(preprocess(i))
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
        print("")
        break
        

Example1:  Create 2D animated sequences for incorporation into animated films, videos, games or other media content
Score: 0.9181568026542664	 Topic: 0.035*"design" + 0.028*"market" + 0.018*"develop" + 0.015*"product" + 0.013*"media"

Example2:  Manage, maintain and grow the sales and relationships with a specific customer or set of accounts. This includes in-depth customer engagement, relationship-building and provision of quality solutions and service to address customers' needs efficiently and generate revenue
Score: 0.9624866247177124	 Topic: 0.022*"busi" + 0.015*"develop" + 0.014*"project" + 0.011*"client" + 0.010*"product"

Example3:  Design accreditation strucures and award learning qualifications based on assessments of alignment with accreditation requirements
Score: 0.756336510181427	 Topic: 0.022*"project" + 0.016*"traineeship" + 0.014*"report" + 0.014*"graduat" + 0.013*"institut"

Example4:  Generate 3D models using a variety of modelling software to represent characteristi

##  Stuff that isn't happening


In [315]:
from nltk.tokenize import word_tokenize
text = "to be or not to be"
# tokens = nltk.word_tokenize(text)
# bigrm = nltk.bigrams(tokens)
# print(*map(' '.join, bigrm), sep=', ')

jobD = [row.split() for row in text]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]

from nltk import ngrams

bigram=list(ngrams(df_bigram['Job Description'],2))

from nltk.util import ngrams

text = "Hi How are you? i am fine and you"

n_grams = ngrams(df_bigram['Job Description'], 2)

for grams in n_grams :
    print(grams)


### Word2Vec


In [326]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [327]:
# from gensim.models import Word2Vec
# wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [328]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

df_w2v = df[['Job Description', 'Industry_unlisted']]
train, test = train_test_split(df_w2v, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['Job Description']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['Job Description']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['Industry_unlisted'])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test.Industry_unlisted))
print(classification_report(test.Industry_unlisted, y_pred,target_names=ind))

In [464]:
!pip install tensorflow

In [ ]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(df) * .7)
train_posts = df_w2v['Job Description'][:train_size]
train_tags = df_w2v['Industry_unlisted'][:train_size]

test_posts = df_w2v['Job Description'][train_size:]
test_tags = df_w2v['Industry_unlisted'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)